### 1. Import Library

In [14]:
import requests
from bs4 import BeautifulSoup
from fpdf import FPDF
import pandas as pd

### 2.Crawl_data_q_a

#### 2.1.Find Elements

In [15]:
# Hàm để cào dữ liệu từ một trang
def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('div', class_='box-item-content')
    articles_data = []
    
    for _, article in enumerate(articles, start=1):
        # Lấy thông tin tác giả và thời gian
        author_info = article.find('div', class_='box-item-top')
        name = author_info.find('span', class_='author').get_text(strip=True)
        time = author_info.find('span', class_='time').get_text(strip=True)

        # Lấy tiêu đề câu hỏi
        question_element = article.find('a', class_='question-title')
        question = question_element.get_text(strip=True)
        
        # Lấy tình huống
        summary = article.find('div', class_='question-sapo').get_text(strip=True)
        
        # Lấy liên kết chi tiết
        detail_link = article.find('a', class_='box-viewmore')['href']
        detail_url = f"https://chinhsachonline.chinhphu.vn{detail_link}"
        
        # Gửi yêu cầu HTTP để lấy nội dung trang chi tiết
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        
        # Lấy câu trả lời từ trang chi tiết
        answer_div = detail_soup.find('div', class_='detail__rcontent')
        answer = answer_div.get_text(strip=True) if answer_div else 'No answer available'
        
        # Thêm thông tin bài viết vào danh sách
        articles_data.append({
            'Name': name,
            'Time': time,
            'Question': question,
            'Situation': summary,
            'Answer': answer
        })
    
    return articles_data


#### 2.2.Execute Function

In [16]:
articles_data = []
for page_num in range(1, 350):  # Thay đổi phạm vi để bao gồm số trang bạn cần
    
    page_url = f"https://chinhsachonline.chinhphu.vn/tai-chinh-ngan-hang-dau-tu-cong-thuong/t1/14/trang-{page_num}.htm"
    articles_data.extend(crawl_page(page_url))
    

#### 2.3 Export csv

In [17]:
# Tạo DataFrame từ danh sách dữ liệu
df_taichinh_nganhang = pd.DataFrame(articles_data)
df_taichinh_nganhang.head(5)

,Name,Time,Question,Situation,Answer
0,Nguyễn Minh Phương,09:22 14/08/2024,Phần chênh lệch thu chi được trích quỹ hay chu...,Tôi công tác tại đơn vị sự nghiệp công lập tự ...,Bộ Tài chính trả lời vấn đề này như sau:Theo q...
1,Trịnh Xuân Quang,08:02 13/08/2024,Các trường hợp không được vay từ quỹ tín dụng ...,Khoản 3 Điều 126 Luật Các tổ chức tín dụng áp ...,Ngân hàng Nhà nước Việt Nam trả lời vấn đề này...
2,Trần Văn Hùng,07:58 13/08/2024,"Ghi nợ tiền sử dụng đất, thủ tục thế nào?","Tôi xin hỏi, theo quy định mới thì trường hợp ...",Tại Điều 22 Nghị định số103/2024/NĐ-CPngày 30/...
3,Nguyễn Thị Thu Minh,16:13 12/08/2024,"Điều kiện hưởng chính sách miễn, giảm tiền sử ...","Tôi xin hỏi, Luật Đất đai 2024 đã có hiệu lực ...","Theo Điều 18, 19 Nghị định số103/2024/NĐ-CPngà..."
4,Nguyễn Nam Hà,14:16 08/08/2024,"Xe sang tên chủ mới, bảo hiểm có còn hiệu lực?",Tôi tham gia bảo hiểm bắt buộc trách nhiệm dân...,"Cục Quản lý, giám sát bảo hiểm, Bộ Tài chính t..."


In [18]:

# Lưu DataFrame vào file CSV
csv_file = '../data/data_tinhhuong/taichinh_nganhang_dautu_congthuong_data.csv'
df_taichinh_nganhang.to_csv(csv_file, encoding='utf-8',index_label='Index')

print(f"Successfully saved data to {csv_file}")

Successfully saved data to ../data/data_tinhhuong/taichinh_nganhang_dautu_congthuong_data.csv


In [19]:
df_taichinh_nganhang=pd.read_csv("../data/data_tinhhuong/taichinh_nganhang_dautu_congthuong_data.csv",encoding='utf-8')

In [20]:
df_taichinh_nganhang.head(5)

,Index,Name,Time,Question,Situation,Answer
0,0,Nguyễn Minh Phương,09:22 14/08/2024,Phần chênh lệch thu chi được trích quỹ hay chu...,Tôi công tác tại đơn vị sự nghiệp công lập tự ...,Bộ Tài chính trả lời vấn đề này như sau:Theo q...
1,1,Trịnh Xuân Quang,08:02 13/08/2024,Các trường hợp không được vay từ quỹ tín dụng ...,Khoản 3 Điều 126 Luật Các tổ chức tín dụng áp ...,Ngân hàng Nhà nước Việt Nam trả lời vấn đề này...
2,2,Trần Văn Hùng,07:58 13/08/2024,"Ghi nợ tiền sử dụng đất, thủ tục thế nào?","Tôi xin hỏi, theo quy định mới thì trường hợp ...",Tại Điều 22 Nghị định số103/2024/NĐ-CPngày 30/...
3,3,Nguyễn Thị Thu Minh,16:13 12/08/2024,"Điều kiện hưởng chính sách miễn, giảm tiền sử ...","Tôi xin hỏi, Luật Đất đai 2024 đã có hiệu lực ...","Theo Điều 18, 19 Nghị định số103/2024/NĐ-CPngà..."
4,4,Nguyễn Nam Hà,14:16 08/08/2024,"Xe sang tên chủ mới, bảo hiểm có còn hiệu lực?",Tôi tham gia bảo hiểm bắt buộc trách nhiệm dân...,"Cục Quản lý, giám sát bảo hiểm, Bộ Tài chính t..."
